<a href="https://colab.research.google.com/github/DaffaHamsy/UTS-SIstem-Cerdas/blob/main/soal_UTS_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import random
import math

# Definisikan koordinat titik-titik pengantaran
customer_coordinates = {
    'a': (15, 5),
    'b': (10, 30),
    'c': (5, 10),
    'd': (15, 20),
    'e': (25, 10),
    'f': (30, 20),
    'g': (45, 5),
    'h': (35, 30),
    'i': (45, 25),
    'j': (40, 10)
}

# Fungsi untuk menghitung jarak antara dua titik
def calculate_distance(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

# Fungsi untuk menghitung total waktu pengantaran dan pendapatan
def calculate_delivery_time_and_income(delivery_sequence):
    total_distance = 0
    total_income = 0
    current_time = 0
    for i in range(len(delivery_sequence) - 1):
        distance = calculate_distance(customer_coordinates[delivery_sequence[i]], customer_coordinates[delivery_sequence[i+1]])
        total_distance += distance
        current_time += distance  # Waktu tempuh dihitung sebagai jarak
        if delivery_sequence[i] == 'a' or delivery_sequence[i] == 'i':
            if current_time <= 180:  # Maksimal 3 jam (180 menit)
                total_income += 15000  # Bonus untuk pelanggan Merah
            else:
                total_income -= 10000 * ((current_time - 180) // 30)  # Denda 10.000 per setiap 30 menit keterlambatan
        elif delivery_sequence[i] == 'e' or delivery_sequence[i] == 'f':
            if current_time <= 120:  # Maksimal 2 jam (120 menit)
                total_income += 15000  # Bonus untuk pelanggan Kuning
            else:
                total_income -= 5000 * ((current_time - 120) // 30)  # Denda 5.000 per setiap 30 menit keterlambatan
    return total_income

# Fungsi untuk menghasilkan populasi awal
def generate_initial_population(population_size, customers):
    initial_population = []
    for _ in range(population_size):
        sequence = customers[:]
        random.shuffle(sequence)  # Acak urutan pengiriman
        initial_population.append(sequence)
    return initial_population

# Fungsi untuk melakukan seleksi berdasarkan nilai fitness
def selection(population, fitness_values):
    selected_population = []
    total_fitness = sum(fitness_values)
    probabilities = [fitness / total_fitness for fitness in fitness_values]
    for _ in range(len(population)):
        selected_population.append(random.choices(population, probabilities)[0])
    return selected_population

# Fungsi crossover untuk menghasilkan generasi baru
def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + [gene for gene in parent2 if gene not in parent1[:crossover_point]]
    child2 = parent2[:crossover_point] + [gene for gene in parent1 if gene not in parent2[:crossover_point]]
    return child1, child2

# Fungsi untuk melakukan mutasi pada kromosom
def mutation(chromosome, mutation_rate):
    mutated_chromosome = chromosome[:]
    if random.random() < mutation_rate:
        gene1, gene2 = random.sample(range(len(chromosome)), 2)
        mutated_chromosome[gene1], mutated_chromosome[gene2] = mutated_chromosome[gene2], mutated_chromosome[gene1]
    return mutated_chromosome

# Fungsi untuk menemukan kromosom terbaik
def find_best_chromosome(population):
    best_chromosome = None
    best_fitness = float('-inf')
    for chromosome in population:
        fitness = calculate_delivery_time_and_income(chromosome)
        if fitness > best_fitness:
            best_chromosome = chromosome
            best_fitness = fitness
    return best_chromosome, best_fitness

# Algoritma genetik untuk optimasi pendapatan abang delivery
def genetic_algorithm(population_size, mutation_rate, num_generations, customers):
    population = generate_initial_population(population_size, customers)
    for generation in range(num_generations):
        fitness_values = []
        for chromosome in population:
            fitness = calculate_delivery_time_and_income(chromosome)
            fitness_values.append(fitness)
        selected_population = selection(population, fitness_values)
        new_population = []
        while len(new_population) < population_size:
            parent1, parent2 = random.sample(selected_population, 2)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutation(child1, mutation_rate)
            child2 = mutation(child2, mutation_rate)
            new_population.extend([child1, child2])
        population = new_population
    best_chromosome, best_fitness = find_best_chromosome(population)
    return best_chromosome, best_fitness

# Main program
population_size = 50
mutation_rate = 0.1
num_generations = 100
customers = ['a', 'b', 'c', 'd', 'e', 'f', 'h', 'i', 'j']  # Exclude 'g'

best_chromosome, best_income = genetic_algorithm(population_size, mutation_rate, num_generations, customers)
best_chromosome.insert(0, 'g')  # Insert 'g' as the starting point
print("Jalur terbaik:", best_chromosome)
print("Total pendapatan:", best_income)


Jalur terbaik: ['g', 'e', 'a', 'h', 'f', 'c', 'i', 'd', 'j', 'b']
Total pendapatan: 60000
